# Introduction to classes

## Code reuse with functions

Here is some simple code to price a European call option by Monte-Carlo simulations:

In [1]:
import numpy as np

T = 5
S_0 = 100.
vol = 0.2
r = 0.02
N = 10000
K = 100

W = np.sqrt(T) * np.random.normal(size=N)
S_T = S_0 * np.exp( (r-0.5*vol**2)*T + vol*W )
C_T = np.exp(-r*T) * np.maximum(S_T-K, 0.)
C_0 = np.average(C_T)
C_0

21.465504355129692

Because we have already learned about functions and we aim to have re-usable code,
we encapsulate the code into a function:

In [2]:
def mc_price_call(spot, rate, vol, maturity, strike, n_paths):
    W = np.sqrt(maturity) * np.random.normal(size=n_paths)
    S_T = spot * np.exp( (rate-0.5*vol**2)*maturity + vol*W )
    V_T = np.exp(-rate*maturity) * np.maximum(S_T-strike, 0.)
    V_0 = np.average(V_T)   

    return V_0

Now we can call our function whenever we need to price a call:

In [3]:
c = mc_price_call(S_0, r, vol, T, K, N)
c

21.635293227077444

How about puts? 

You could just copy the function, give it a new name and change just the payoff
calculation from `np.maximum(S_T-strike, 0.)` to `np.maximum(strike-K, 0.)`

In [4]:
def mc_price_put(spot, rate, vol, maturity, strike, n_paths):
    W = np.sqrt(maturity) * np.random.normal(size=n_paths)
    S_T = spot * np.exp( (rate-0.5*vol**2)*maturity + vol*W ) 
    # Put payoff    
    V_T = np.exp(-rate*maturity) * np.maximum(strike-S_T, 0.) 
    V_0 = np.average(V_T)   

    return V_0

This works:

In [5]:
p = mc_price_put(S_0, r, vol, T, K, N)
p

12.38837082223212

but is very unsatisfactory. The whole point about functions is to reuse code, but ended up
creating a whole function which was almost a copy of the previous one except for swapping
the payoff calculation.

It would seem that you would want to parametrize the function with an argument to calculate the payoff, so that it can vary based on what option you want to price.

So we instead create a function that takes a `payoff` argument which we use as a function to
calculate the payoff on each path by invoking it, i.e. `payoff(S_T)`

In [6]:
def mc_price_european(spot, rate, vol, maturity, strike, n_paths, payoff):
    W = np.sqrt(maturity) * np.random.normal(size=n_paths)
    S_T = spot * np.exp( (rate-0.5*vol**2)*maturity + vol*W )     
    V_T = np.exp(-rate*maturity) * payoff(S_T)
    V_0 = np.average(V_T)   

    return V_0

We can now use the new function to implement the call one

In [7]:
def mc_price_call(spot, rate, vol, maturity, strike, n_paths):
    def payoff_function(S):
        return np.maximum(S-strike, 0.)
    
    return mc_price_european(spot, rate, vol, maturity, strike, n_paths, payoff_function)

c = mc_price_call(S_0, r, vol, T, K, N)
c

22.264585540324845

Note, how we were able to define a local function inside another function. 

Let's also implement the put pricing function:

In [8]:
def mc_price_put(spot, rate, vol, maturity, strike, n_paths):
    p = lambda x : np.maximum(strike-x, 0.)
    return mc_price_european(spot, rate, vol, maturity, strike, n_paths, p)

p = mc_price_put(S_0, r, vol, T, K, N)
p

12.58559890168766

Note how this time we used a `lambda` or anonymous function instead of defining a named function.

We can use that to actually call our `mc_price_european` function with arbitray payoffs
created on the fly when invoking the function:

In [9]:
z = mc_price_european(S_0, r, vol, T, K, N, lambda x: np.sqrt(x))
z

9.267292694517682

## First look at classes

If you think about what the payoff of an option is, you would come to the conclusion
that the `strike` is an attribute of the payoff, and the maturity and the payoff
itself are attributes of the option. Furthermore, a payoff is an object that can calculate
the value of the payoff scenarios given scenarios of the underlying at maturity.

To represent types that have attributes and behavior, we need to use the `class` keyword.
Classes allow you to define your own types with their attributes and behaviors.

In [10]:
# Hide code 

class CallPayoff:  # define a class object

    # define a constructor taking the strike
    # self is the instance object being constructed
    def __init__(self, strike):
        self.strike = strike  # instance object strike attribute set by constructor


    # define calculate method
    def calculate(self, spot):
        return np.maximum(spot - self.strike, 0.)

Let's see how you would use your new `CallPayoff` type  for example.

You would construct or initialize it with a strike value

In [11]:
call_payoff = CallPayoff(K)

then you could access its strike attribute

In [12]:
call_payoff.strike

100

similarly you would invoke its `calculate` method, passing to it the scenarios of the 
underlying at expiry and it would return the value of the payoff

In [13]:
pay = call_payoff.calculate(120.)
pay

20.0

This is the type of the object

In [14]:
type(call_payoff)

__main__.CallPayoff

This is how you would define the class to get the behavior above:

In [15]:
class CallPayoff:  # define a class object

    # define a constructor taking the strike
    # self is the instance object being constructed
    def __init__(self, strike):
        self.strike = strike  # instance object strike attribute set by constructor


    # define calculate method
    def calculate(self, spot):
        return np.maximum(spot - self.strike, 0.)

When we invoke `print` on a python object, we usually get some meaningful string,
for example

In [16]:
a = [ 1., 2., 3.]
a

[1.0, 2.0, 3.0]

In [17]:
b = np.array(a)
b

array([1., 2., 3.])

In [18]:
print(a)
print(b)

[1.0, 2.0, 3.0]
[1. 2. 3.]


Let's try that with our type

In [19]:
print(call_payoff)

In [20]:
call_payoff

Let's make that more meaningful by adding some special Python operators to our class:

In [21]:

class CallPayoff:

    def __init__(self, strike):
        self.strike = strike

    def calculate(self, spot):
        return np.maximum(spot - self.strike, 0.)

    def __repr__(self):
        return 'CallPayoff({0})'.format(self.strike)        
    
    def __str__(self):
        return 'European call payoff with strike {0}'.format(self.strike)        

`__repr__()` is meant to return a string representation of the object and is called by the 
built-in `repr()` function or when evaluating objects, for example:

In [22]:
call_payoff = CallPayoff(K)
repr(call_payoff)

'CallPayoff(100)'

In [23]:
call_payoff

CallPayoff(100)

`__str__()` also returns a string representation destined to print output and be readable for users. It is called by the built-in `str()` and `print()` functions

In [24]:
print(call_payoff)
str(call_payoff)

European call payoff with strike 100


'European call payoff with strike 100'

To use our class with the Monte-Carlo function would not immediately work, because the Monte-Carlo function expects a callable object that is invoked like `call_payoff(S_T)`.
Although, our class has a payoff calculation method, it is invoked with the syntax
`call_payoff.calculate(S_T)`. We could change the Monte-Carlo to use this syntax, but 
Python also defines a callable operator that you can implement for your classes and it will be
invoked whenever the function call syntax is used. The operator is `__call__()` and you define
it in your class, object instance created from the class will behave like functions when
they are invoked with function call syntax:

In [25]:
class CallPayoff:

    def __init__(self, strike):
        self.strike = strike

    def __call__(self, spot):
        return np.maximum(spot - self.strike, 0.)

We can now have a new version of the Monte-Carlo function that does not need
to take the strike as an argument since it is an attribute of the payoff:

In [26]:
def mc_price_european(spot, rate, vol, maturity, n_paths, payoff):
    W = np.sqrt(maturity) * np.random.normal(size=n_paths)
    S_T = spot * np.exp( (rate-0.5*vol**2)*maturity + vol*W )     
    V_T = np.exp(-rate*maturity) * payoff(S_T)
    V_0 = np.average(V_T)   

    return V_0

Let's try it

In [27]:
call_payoff = CallPayoff(K)
z = mc_price_european(S_0, r, vol, T, N, call_payoff)
z

21.94320686495762